In [1]:
from qiskit import *
from qiskit.tools.jupyter import *
from qiskit.visualization import *

from qiskit_ibm_runtime import QiskitRuntimeService
 
# Save an IBM Quantum account and set it as your default account.
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="0522c2a92bd309b3c7fdf9a4ee6a722dd282493f04c9aa2852a38dfbf5f0f4fcbbef3289c43907c0e0145566d79021a1c336e30a689fe727820e3f77b6e13b31", set_as_default=True)
 
# Load saved credentials
service = QiskitRuntimeService()

from qiskit_ibm_provider import IBMProvider
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from qiskit_experiments.library import T1, T2Ramsey, T2Hahn
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment
from qiskit import schedule, transpile

In [3]:
def T1_T2Hahn_batch(min_delay, max_delay, num_samples, num_echos, shots, backend, physical_qubits, num_batches):
     # min_delay = 100e-9
     # max_delay = 400e-6
     # num_samples = 200
    delays = np.geomspace(min_delay, max_delay, num_samples)
    
    t1 = [T1(physical_qubits = [qubit],
             delays = delays,
             backend = backend) for qubit in physical_qubits]
    
    t2 = [T2Hahn(physical_qubits = [qubit],
             delays = delays,
             backend = backend,
             num_echoes = num_echos) for qubit in physical_qubits]
    
    for i in range(len(physical_qubits)):
        t1[i].set_run_options(shots=shots)
        t2[i].set_run_options(shots=shots)
    
    t1_par = ParallelExperiment(t1,backend=backend,flatten_results=False)
    t2_par = ParallelExperiment(t2,backend=backend,flatten_results=False)
    
    t1_par.set_run_options(shots=shots)
    t2_par.set_run_options(shots=shots)
    
    batch = []
    for i in range(num_batches):
        
        batch.append(t1_par)
        batch.append(t2_par)
    batch_exp = BatchExperiment(batch, backend= backend, flatten_results= False)
    batch_exp.set_run_options(shots=shots)
    batch_result = batch_exp.run(backend=backend).block_for_results()
    return batch_exp, batch_result

In [4]:
# from qiskit.providers.fake_provider import FakeKolkata
instance = 'ibm-q/open/main'
provider = IBMProvider(instance=instance)

provider_osaka = provider.get_backend('ibm_osaka')
backend = provider_osaka
shots = 2000
max_delay = 400e-6
min_delay = 100e-9
num_samples = 150
num_batches = 10
num_echos = 1

config_backend = backend.configuration()

props_backend = backend.properties()

physical_qubits = [6, 10, 13, 15, 20, 21, 22] # the no crosstalk qubits selected from previous experiment   

T1_value = [props_backend.t1(i) for i in physical_qubits]
T2_value = [props_backend.t2(i) for i in physical_qubits]

print(T1_value)
print(T2_value)

[0.00027943191129357436, 0.00022364709645466668, 0.00013543184573280972, 0.00032098897692979425, 0.00041188278097099775, 0.00037890944781083705, 0.00037596521763143265]
[0.00027970521698336313, 0.0003576078273948061, 7.129215363668975e-05, 0.00028231382328281206, 0.0001651396978084268, 0.00014386313646303346, 0.00023153477784397088]


In [5]:
batch_exp, batch_result = T1_T2Hahn_batch(min_delay=min_delay,
                                          max_delay=max_delay,
                                          physical_qubits=physical_qubits,
                                          num_echos=num_echos,
                                          num_batches=num_batches,
                                          backend=backend,
                                          shots = shots,
                                          num_samples=num_samples)

IBMBackendApiError: 'Error submitting job: \'409 Client Error: Conflict for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"You have reached the limit of 3 pending  jobs. Please wait for a job to complete or cancel one before submitting anything new.","code":3458,"solution":"Wait until some previous jobs were finished. You can cancel pending jobs to run new jobs.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''